[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bachvu98/Policy-NLP/blob/master/Model_Development_Test.ipynb)


In [ ]:
from os import listdir
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import *
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from lxml import etree, html
import json
import time


In [ ]:
sites = pd.read_csv('D:/Project/Privacy Policy NLP/Policy-NLP/OPP-115_v1_0/OPP-115/documentation/websites_covered_opp115.csv',index_col=3,parse_dates=[4])
sites.head()

,Site UID,Site URL,Site Human-Readable Name,Site Check Date,In 115 Set?,Comments,Sectoral Data,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106
Policy UID,,,,,,,,,,,,,,,,,,,,,
20,1,theatlantic.com,The Atlantic,2016-02-08,Yes,"Alexa Rank: 975 (Global), 289 (US)","Arts: Literature: Authors: A: Alcott, Louisa M...","Arts: Literature: Authors: M: Merwin, W. S.: W...",Arts: Literature: World Literature: American: ...,"Arts: Music: Composition: Composers: T: Toch, ...",...,Regional: North America: United States: Tennes...,Regional: North America: United States: Vermon...,Regional: North America: United States: Washin...,Regional: Oceania: Australia: New South Wales:...,Science: Agriculture: Crop Plants: Coffee,Science: Astronomy: Solar System: Dwarf Planet...,Science: Biology: Microbiology: Virology,Science: Earth Sciences: Atmospheric Sciences:...,Science: Social Sciences: Economics: Economic ...,Science: Social Sciences: Economics: People: G...
21,2,imdb.com,IMDb,2016-02-08,Yes,"Alexa Rank: 49 (Global), 27 (US)",Kids and Teens: Entertainment: Animation: Movies,Regional: Europe: United Kingdom: Arts and Ent...,Arts: Performing Arts: Magic: Magicians: Siegf...,Arts: Television: Programs: Science Fiction an...,...,"Arts: People: S: Sagal, Jean and Liz","Arts: People: W: Wilde, Lyn and Lee",Arts: Performing Arts: Magic: Magicians: Penn ...,Arts: Performing Arts: Puppetry: Muppets: Movi...,Arts: Performing Arts: Stunts: Players,Arts: Performing Arts: Theatre: Awards: Americ...,Arts: Television: Programs: Action and Adventu...,Arts: Television: Programs: Action and Adventu...,Arts: Television: Programs: Children's: Beetle...,Arts: Television: Programs: Comedy: Sitcoms: W...
26,3,nytimes.com,New York Times,2016-02-08,Yes,"Alexa Rank: 101 (Global), 22 (US)",Society: Issues: Warfare and Conflict: Specifi...,Science: Social Sciences: Psychology: Evolutio...,Society: Issues: Warfare and Conflict: Specifi...,Computers: E-Books: News and Media,...,Science: Technology: Space: Missions: Manned: ...,Society: Activism: Media: Culture Jamming: Spo...,Society: Crime: Sex Offenses: Child Pornograph...,Society: History: By Topic: Maritime: Ships: O...,"Society: History: Historians: Parker, Geoffrey",Society: Issues: Business: Allegedly Unethical...,Society: Issues: Education: Standardized Testing,Society: Issues: Environment: Light Pollution:...,Society: Issues: Environment: News and Media: ...,Society: Issues: Environment: News and Media: ...
32,4,theverge.com,The Verge,2016-02-08,Yes,"Alexa Rank: 525 (Global), 230 (US)",Home: Consumer Information: Computers and Inte...,Home: Consumer Information: Electronics: Weblogs,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,5,nbc.com,NBC Universal,2016-02-08,Yes,"Alexa Rank: 1548 (Global), 426 (US)",Kids and Teens: Entertainment: Television: Rea...,Arts: Television: Networks: NBC,Arts: Television: Programs: Action and Adventu...,Arts: Television: Programs: Comedy: Sitcoms: 3...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Change interpretation of 115 set membership column
sites['In 115 Set?']=sites['In 115 Set?'].apply(lambda yn: True if yn == 'Yes' else False)

In [ ]:
#Reinterpret the categories for the sites. Find the primary category and take the mode across all columns
sectors = sites[sites.columns[6:]]
sectors = sectors.applymap(lambda s: str(s).split(':')[0])
sectors.head()

,Sectoral Data,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,...,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106
Policy UID,,,,,,,,,,,,,,,,,,,,,
20,Arts,Arts,Arts,Arts,Arts,Arts,Regional,Regional,Science,Society,...,Regional,Regional,Regional,Regional,Science,Science,Science,Science,Science,Science
21,Kids and Teens,Regional,Arts,Arts,Arts,Arts,Arts,Arts,Regional,Sports,...,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts
26,Society,Science,Society,Computers,Arts,Arts,Arts,Society,Regional,Society,...,Science,Society,Society,Society,Society,Society,Society,Society,Society,Society
32,Home,Home,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
33,Kids and Teens,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [ ]:
s = []
for i in range(0,sectors.shape[0]):
    
    sec = sectors.iloc[i][sectors.iloc[i]!='nan'].mode()
    sec = sec.iloc[0] if len(sec) > 0 else 'None'
    s.append(sec)

In [ ]:
#Drop and append
sites = sites.drop(sites.columns[6:], axis=1)
sites['Sector'] = s

In [ ]:
#Grab the policies table
policies = pd.read_csv('D:/Project/Privacy Policy NLP/Policy-NLP/OPP-115_v1_0/OPP-115/documentation/policies_opp115.csv',index_col=0,parse_dates=[2,3])
policies = policies.drop('Unnamed: 4',axis=1)
policies.head()

,Policy URL,Policy collection date,Policy last updated date
Policy UID,,,
20,theatlantic.com/privacy-policy/,2015-07-02,2015-01-01
21,imdb.com/privacy,2015-07-02,2014-12-05
26,nytimes.com/privacy,2015-07-08,2015-06-10
32,voxmedia.com/privacy-policy,2015-07-02,2014-05-01
33,nbcuniversal.com/privacy/full-privacy-policy,2015-07-02,2015-01-14


In [ ]:
base_dir = 'D:/Project/Privacy Policy NLP/Policy-NLP/OPP-115_v1_0/OPP-115/sanitized_policies/'
files = [f for f in listdir(base_dir) if f.endswith('.html')]

In [ ]:
#Check matching policy ids
pids = sites.index.tolist()
for f in files:
    pid = np.int(f.split('_')[0])
    if pid not in pids:
        print('MISSING '+str(pid))

In [ ]:
txt = []
pids = []
for f in files:
    with open(base_dir + f, 'r') as pg:
        #print('Reading: ' + f)
        page = pg.read()
        txt.append(page)
        pid = np.int(f.split('_')[0])
        pids.append(pid)

In [ ]:
#Create dataframe of texts and join it to policies frame
txts = pd.DataFrame({'policy_text': txt},index=pids)
txts.index.name = 'Policy UID'
policies = pd.merge(policies,txts,left_index=True,right_index=True,how='outer')

In [ ]:
#Now merge the sites and policies tables
sites = pd.merge(sites,policies,left_index=True,right_index=True,how='outer')
sites.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 191 entries, 20 to 1713
Data columns (total 11 columns):
Site UID                    191 non-null int64
Site URL                    191 non-null object
Site Human-Readable Name    191 non-null object
Site Check Date             191 non-null datetime64[ns]
In 115 Set?                 191 non-null bool
Comments                    191 non-null object
Sector                      191 non-null object
Policy URL                  191 non-null object
Policy collection date      191 non-null datetime64[ns]
Policy last updated date    159 non-null datetime64[ns]
policy_text                 191 non-null object
dtypes: bool(1), datetime64[ns](3), int64(1), object(6)
memory usage: 16.6+ KB


In [ ]:
sites.index.nunique()

115

In [ ]:
sites.to_csv("D:/Project/Privacy Policy NLP/Policy-NLP/OPP-115_v1_0/OPP-115/sites.csv")

In [ ]:
url = "D:/Project/Privacy Policy NLP/Policy-NLP/OPP-115_v1_0/OPP-115/annotations/"
files = [f for f in listdir(url) if f.endswith('.csv')]

In [ ]:
names = """annotation_id 
        batch_id 
        annotator_id 
        policy_id 
        segment_id 
        category_name 
        attributes_value_pairs 
        date 
        policy_url""".split()

types = {'annotation_id': np.int,
        'batch_id': str,
        'annotator_id': np.int,
        'policy_id': np.int,
        'segment_id': np.int,
        'category_name': str}

ann_list = []
pids = []
for f in files:
    df = pd.read_csv(url + f,header=None,names=names,na_values={'date': 'Not specified'},
                 parse_dates=[7],index_col=0)
    ann_list.append(df)
    pids.append(np.int(f.split('_')[0]))

In [ ]:
annotations = pd.concat(ann_list,axis=0,keys=pids,names=['Policy UID','annotation_id'])
annotations = annotations.drop('policy_id',axis=1)

In [ ]:
annotations.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 23194 entries, (1017, 20137) to (995, 6754)
Data columns (total 7 columns):
batch_id                  23194 non-null object
annotator_id              23194 non-null int64
segment_id                23194 non-null int64
category_name             23194 non-null object
attributes_value_pairs    23194 non-null object
date                      19460 non-null datetime64[ns]
policy_url                23194 non-null object
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 1.5+ MB


In [ ]:
attr_values = pd.DataFrame(data=None,columns=['annotation_id','start_idx','end_idx','attribute','text','value'])
template = dict.fromkeys('startIndexInSegment endIndexInSegment selectedText value'.split())

In [ ]:
#BATCH PROCESSING OF ATTRIBUTE-VALUE PAIRS: THIS NEEDS TO BE DONE MANUALLY!
attr_values1 = pd.DataFrame(data=None,columns=['annotation_id','start_idx','end_idx','attribute','text','value'])
t0 = time.time()
print('Starting at ' + str(t0))
for i in range(0,annotations.shape[0]):    #Note manual adjustment of batch size here...
    
    attr_val = annotations['attributes_value_pairs'].iloc[i]
    ann_id = annotations.index.values[i][-1]
    
    obj = json.loads(attr_val)
    keys = list(obj.keys())
    
    for k in keys:
        obj2 = dict(template, **obj[k]) #Ensures at a minimum we get the empty template data.
        df = pd.DataFrame({'annotation_id': [ann_id],
                           'start_idx': [obj2['startIndexInSegment']],
                           'end_idx': [obj2['endIndexInSegment']],
                           'attribute': [k],
                           'text': [obj2['selectedText']],
                           'value': [obj2['value']]})
        attr_values1 = attr_values1.append(df,ignore_index=True)
t1 = time.time()
print('Finished at ' + str(t1))
print('Total elapsed time ' + str(t1-t0))
attr_values = attr_values.append(attr_values1,ignore_index=True)
attr_values.info()

Starting at 1596478307.2138872


In [ ]:
#Set up indices
attr_values.index.name = 'text_id'
attr_values = attr_values.set_index(['annotation_id'], append=True).reorder_levels(['annotation_id','text_id'])

In [ ]:
annotations.to_csv("D:/Project/Privacy Policy NLP/Policy-NLP/OPP-115_v1_0/OPP-115/annotations.csv")
attr_values.to_csv("D:/Project/Privacy Policy NLP/Policy-NLP/OPP-115_v1_0/OPP-115/text_selected.csv")

In [ ]:
#First grab policy text from the sites table to parse down
policies = pd.DataFrame(sites['policy_text'])
policies.head()

In [ ]:
#For each policy, create a dataframe of segments with unique segment index
dfs = []
for i in range(0,policies.shape[0]):
    tmp = pd.DataFrame(policies['policy_text'].iloc[i].split('|||'),columns=['segments'])
    tmp['Policy UID'] = policies.index.values[i]
    tmp.index.name = 'segment_id'
    dfs.append(tmp)

In [ ]:
segments = pd.concat(dfs,axis=0).reset_index().set_index(['Policy UID','segment_id'])

In [ ]:
segments.to_csv("D:/Project/Privacy Policy NLP/Policy-NLP/OPP-115_v1_0/OPP-115/segments.csv")